In [1]:
import random

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
import numpy as np

In [5]:
arr = np.array([[1,2,0], [3,4,0], [4,5,0]])

In [7]:
arr

array([[1, 2, 0],
       [3, 4, 0],
       [4, 5, 0]])

In [6]:
np.where(arr == 0)

(array([0, 1, 2]), array([2, 2, 2]))